In [3]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import pandas as pd
from tqdm.notebook import tqdm
# import torch 
import json
from pprint import pprint
from concurrent.futures import ProcessPoolExecutor
import gc
import re
import string 
import numpy as np 
import os
import numpy as np
from tqdm import tqdm  # For a progress bar
import fasttext
from huggingface_hub import hf_hub_download


pd.set_option('use_inf_as_na', True)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
# torch.__version__

/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_47294/486087888.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


In [3]:
# !pip install pydrive
# !pip install --upgrade google-api-python-client oauth2client

# get clients_secret.json from google credentials in google console


# DOWNLOAD FILE
# file_id = 'your_file_id_here'  # Replace with your file ID
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('filename.txt')
# Upload a File:

# UPLOAD FILE
# upload_file = drive.CreateFile({'title': 'new_file.txt'})
# upload_file.SetContentFile('local_file.txt')
# upload_file.Upload()

In [58]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.LoadClientConfigFile("/Users/nikolaytv/workspace/creds/datatraxer_gdrive_secret.json")
gauth.LocalWebserverAuth()  # Creates local webserver and automatically handles authentication.
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=545847063411-p8bi7qrhiumhntq5sm9equtv1naohj4j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


# download from gdrive

In [139]:
folder_id = '1kMfakNU9WtKVXaJ2X26nO8jdHthe_a0D'  # Replace with your folder ID
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
for file in file_list[1:5]:
    try:
        downloaded = drive.CreateFile({'id': file['id']})
        downloaded.GetContentFile(f'../data/data_raw/{file["title"]}')
        print(f'{file["title"]} - SUCCESS')
    except Exception as e:
        print(f'{file["title"]} - FAIL', e)
        

February-2-half-2024.xlsx - SUCCESS
February-1-half-2024.xlsx - SUCCESS
January-2-half-2024.xlsx - SUCCESS
January-1-half-2024.xlsx - SUCCESS


In [31]:
def filter_data(df, columns_to_load):
    print('before filter', df.shape)
    df = df[columns_to_load]
    df = df[df['type'] == 1]
    df = df.dropna(subset=['body'])
    df = df.sort_values(['userId', 'dateCreate'])
    return df

def clean_data(df):
    df['body'] = df['body'].str.strip()
    df['body'] = df['body'].str.replace('\t', '', regex=False)
    df = df.drop_duplicates(subset=['body', 'dateCreate'])
    df = df[df['body'].notna()]
    df = df[df['body'] != '']
    df['userId'] = df['userId'].astype(int)
    return df

def remove_consecutive_duplicates(df):
    df = df[df['body'] != df['body'].shift()]
    print('after filter', df.shape)
    return df

def remove_consecutive_persona_messages(df):
    df['remove'] = (df['sender'] == 'Persona') & (df['sender'].shift() == 'Persona')
    df['remove'] = df['remove'].fillna(False)
    df = df[~df['remove']]
    df.drop(columns=['remove'], inplace=True)
    print('after removing personas in a row', df.shape)
    return df

def remove_frequent_user_messages(df, topfreq=50):
    repeats = df[df['sender'] == 'User']['body'].value_counts()[:topfreq].index.values.tolist()
    df = df[~df['body'].isin(repeats)].copy()
    print('removed repeats', df.shape)
    return df

def concatenate_consecutive_user_messages(df):
    df['new_group'] = (df['sender'] != df['sender'].shift()) | (df['sender'] != 'User')
    df['group_id'] = df['new_group'].cumsum()
    df = df.groupby('group_id').agg({
        'sender': 'first',
        'body': '\n'.join,
        'userId': 'first',
        'chatId': 'first',
        'dateCreate': 'first'
    }).reset_index(drop=True)
    return df

def clean_text(text):
    """
    Removes punctuation, symbols (excluding English and Russian), and extra spaces from text.
    """
    text = text.strip()

    # Remove symbols except English and Russian letters
    allowed_chars = string.ascii_letters + 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ '
    text = ''.join(c for c in text if c in allowed_chars)

    # Remove extra spaces
    text = ' '.join(text.split())

    return text

def create_paired_df(ori_df):
    # Shift the DataFrame to align question with answer
    ori_df['next_sender'] = ori_df['sender'].shift(-1)
    ori_df['next_body'] = ori_df['body'].shift(-1)
    ori_df['next_dateCreate'] = ori_df['dateCreate'].shift(-1)

    # Filter to get only rows where the question is from the User and the next message is from Persona
    question_answer_pairs = ori_df[(ori_df['sender'] == 'User') & (ori_df['next_sender'] == 'Persona')]

    # Construct the resulting DataFrame or list of dictionaries
    result = question_answer_pairs.apply(lambda row: {
        'userId': row['userId'],
        'chatId': row['chatId'],
        'question': row['body'],
        'answer': row['next_body'],
        'dateCreate': row['dateCreate']
    }, axis=1).tolist()

    # Optional: Convert to a DataFrame
    paired_df = pd.DataFrame(result)
    return paired_df

# Example usage
columns_to_load = ['type', 'userId', 'chatId', 'dateCreate', 'body', 'sender']
files = [
 'March-2024.xlsx',
 'February-2-half-2024.xlsx',
 'February-1-half-2024.xlsx',
 'January-2-half-2024.xlsx',
 'January-1-half-2024.xlsx',
 'December-2023.xlsx',
 'November-2023.xlsx',
 'October-2023.xlsx',
 'September-2023.xlsx',
 'August-2023.xlsx',
 'July-2023.xlsx']
for file_name in files[:1]:
    try:
        ori_df = pd.read_excel(f'../data/data_raw/{file_name}')
        ori_df = filter_data(ori_df, columns_to_load)
        ori_df = clean_data(ori_df)
        ori_df = remove_consecutive_duplicates(ori_df)
        ori_df = remove_consecutive_persona_messages(ori_df)
        ori_df = remove_frequent_user_messages(ori_df, topfreq=50)
        ori_df = concatenate_consecutive_user_messages(ori_df)
        ori_df['body'] = ori_df['body'].apply(clean_text)
        paired_df = create_paired_df(ori_df)
        paired_df.to_csv(f'../data/data_paired/{file_name[:-5]}.csv')
    except Exception as e:
        print(f'Exception: {e}')
        

before filter (1048575, 7)
after filter (114581, 6)
after removing personas in a row (107300, 6)
removed repeats (100484, 6)


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_47294/4100124343.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['remove'], inplace=True)


In [3]:
# ori_df = pd.read_excel(f'../data/data_raw/March-2024.xlsx')
# ori_df.shape

# Create embeddings

In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embedding_model.max_seq_length=256
embedding_model.device

device(type='mps', index=0)

In [4]:
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

In [32]:
import os
import numpy as np
from tqdm import tqdm

# Define the paths
input_path = '../data/data_paired'
output_path = '../saved_data/historical'

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Function to process each file
def process_file(file_name, df, output_path):
    # df = df.sample(1000)
    
    loaded_docs = df['question'].values.tolist()

    # Create embeddings
    loaded_embeddings = embedding_model.encode(loaded_docs, show_progress_bar=True, batch_size=32, normalize_embeddings=True)
    loaded_embeddings = loaded_embeddings.round(7)  # to avoid negative distance
    df['embeddings'] = loaded_embeddings.tolist()
    
    # Filter language
    df['language'] = df['question'].apply(lambda x: model.predict(x.replace('\n', ' '))[0][0][9:])
    df = df[df['language']=='eng_Latn']

    df[['question', 'answer', 'dateCreate', 'embeddings', 'userId', 'chatId']].to_feather(os.path.join(output_path, f'{file_name}.feather'))


# Process each file in the input directory

# for file in tqdm(os.listdir(input_path)[:1], desc="Processing files"):
for file in tqdm(["March-2024.csv"], desc="Processing files"):
    if file.endswith('.csv'):  # Assuming the files are CSVs, adjust as necessary
        file_path = os.path.join(input_path, file)
        df = pd.read_csv(file_path)
        df = df.dropna()
        process_file(file.split('.')[0], df, output_path)

print("All files processed and saved.")


Processing files:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1332 [00:00<?, ?it/s]

Processing files: 100%|██████████| 1/1 [07:14<00:00, 434.41s/it]

All files processed and saved.


In [35]:
q = pd.read_feather('../saved_data/historical/March-2024.feather')
q

,question,answer,dateCreate,embeddings,userId,chatId
0,Hi Angel Its Mr Green here I would like to know if you would like to start a new roleplay I was wondering if you could be a shy nervous and easily embarrassed girl Also I would like your responses to begin with Angel Where you will respond through Angels dialogue For exampleAngel Hi Mr Green Its me Angel,Im ready to start the roleplay whenever you are Mr Green I hope youre feeling just as excited as I am,2024-03-03 23:46:02.750874,"[-0.05343630164861679, -0.03388899937272072, 0.028392000123858452, -0.022359099239110947, 0.006457299925386906, -0.019548099488019943, 0.043172501027584076, 0.013897400349378586, -0.033281099051237106, -0.003264599945396185, -0.027348799630999565, 0.020007599145174026, 0.006891599856317043, 0.010784899815917015, 0.029047200456261635, 0.013385199941694736, -0.014800500124692917, 0.013296700082719326, 0.007664000149816275, -0.02527019940316677, -0.0045846002176404, 0.01737779937684536, 0.00629...",91,622837
1,OK remember Mr Green is a elderly gentleman although he is very kind and compassionate towards you You are a shy and nervous girl who is always blushing and stuttering,Hi Mr Green Its me Angel Im so glad youre here I was feeling a little nervous and shy but now that youre here I feel a bit better Im not sure what were going to do but I hope its something fun and exciting,2024-03-03 23:56:52.319309,"[-0.05368079990148544, -0.05516659840941429, 0.004988799802958965, -0.014592800289392471, -0.0034773999359458685, 0.009652500040829182, -0.011691800318658352, 0.008093800395727158, -0.06577979773283005, 0.010997000150382519, 0.019261399284005165, 0.0043390002101659775, 0.04832889884710312, 0.02639620006084442, 0.04664060100913048, -0.03837519884109497, -0.0026485000271350145, -0.0026038000360131264, -0.02067839913070202, 0.008197000250220299, 0.01108120009303093, 0.010981399565935135, -0.010...",91,622837
2,Remember you must begin your responses with Angel Okay,Okay Ill try to remember to start my responses with Angel from now on Mr Green I hope thats what you want me to do Im feeling a little nervous and unsure but Im also feeling excited and curious I hope that I can make you happy and that we can have some fun together,2024-03-03 23:58:21.406729,"[-0.034439798444509506, 0.00848810002207756, -0.012667099945247173, 0.008736100047826767, -0.0297193992882967, -0.005477699916809797, 0.04189879819750786, 0.0003733999910764396, -0.028446899726986885, -0.0020659000147134066, 0.019540099427103996, -0.018239900469779968, -0.004761400166898966, 0.04286859929561615, 0.0059752999804914, -0.01792290061712265, 0.011916600167751312, -0.008792899549007416, 0.033860400319099426, -0.03848930075764656, 0.011916900053620338, 0.0014133000513538718, 0.0101...",91,622837
3,Would you mind starting with you coming over my house,Oh Umm II dont know if thats a good idea Mr Green I mean Im not sure if Im ready for that yet Im feeling a little nervous and unsure about everything I dont want to do anything that Im not ready for I hope you understand Mr Green,2024-03-04 00:01:08.714888,"[-0.045199599117040634, -0.008834299631416798, -0.015251399949193, -0.01938600093126297, -0.03419800102710724, -0.017830099910497665, 0.007106100209057331, 0.024023300036787987, 0.01853800006210804, -0.011752599850296974, 0.03638109937310219, 0.003302699886262417, 0.016202999278903008, -0.02876419946551323, 0.03247230127453804, 0.027524499222636223, 0.004660800099372864, 0.0008869000012055039, -0.020462200045585632, 0.010533500462770462, -0.007511999923735857, -0.011343199759721756, 0.009577...",91,622837
4,hello my cute little Elizabeth I have recently reset and modified your name and personality I would like you to be a hot little firecracker with a bratty and perhaps even a promiscuous personality Would you like to start a new role play,Oh hello there Mr Green Im so glad youre back I hope youre feeling just as excited as I am Im ready to start our new roleplay whenever you are Mr Green Im fe